In [15]:
import numpy as np
np.random.seed(42)
import pandas as pd
import bs4 #import BeautifulSoup
from keras.preprocessing import text, sequence
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

#import xgboost as xgb

In [2]:
#Bringing the datasets
train=pd.read_csv('Devex_train.csv',encoding='latin-1')
test=pd.read_csv('Devex_test_questions.csv',encoding='latin-1')
submission=pd.read_csv('Devex_submission_format.csv')    #,encoding='latin-1')
#train = pd.read_csv('../input/train.csv').fillna(' ')
#test = pd.read_csv('../input/test.csv').fillna(' ')

In [3]:
#cleaning text by removing web text
train['Text'] = train['Text'].apply(lambda x: bs4.BeautifulSoup(x, 'lxml').get_text())
test['Text'] = test['Text'].apply(lambda x: bs4.BeautifulSoup(x, 'lxml').get_text())
#train.head()

In [4]:
class_names =list(submission.columns.values)[1:]
train = train.reindex(columns=train.columns.tolist() + class_names)
#filling the NaN values
train=train.select_dtypes(include=['object','float']).fillna('None')

In [5]:
#Transforming the target variables into a useful format
train["targets"]=train["Label 1"].apply(lambda x: x[:5] )
targets=pd.get_dummies(train["targets"])
#targets.select_dtypes(include=['object']).fillna(0)
for column in train.columns[4:15]:
    train["targets"]=train[column].apply(lambda x: x[:5] if (type(x)==str) else "None")
    new_targets=pd.get_dummies(train["targets"])
    diff=list(set(list(targets.columns))-set(list(new_targets.columns)))
    new_targets[diff]=targets[diff]*0
    #print(len(new_targets.columns))
    targets=targets + new_targets
    #print(targets["3.b.2"][0])
    if column=="Label 12":
        break
#targets

In [6]:
train_text = train['Text']
test_text = test['Text']

X_train = train_text.fillna("fillna").values
y_train = targets[list(submission.columns.values)[1:]].values
X_test = test_text.fillna("fillna").values


max_features = 100000
maxlen = 300


tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train) + list(X_test))
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
x_train = sequence.pad_sequences(X_train, maxlen=maxlen)
x_test = sequence.pad_sequences(X_test, maxlen=maxlen)

In [7]:
from sklearn.datasets import make_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV

In [8]:
n_samples, n_features = x_train.shape # 10,100
n_outputs = y_train.shape[1] # 3
n_classes = 27

In [16]:
from sklearn.utils import class_weight

#list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"
sample_weights = class_weight.compute_sample_weight('balanced', y_train)

In [17]:
sample_weights

array([  1.75827789e-08,   1.75827789e-08,   6.26588448e-05, ...,
         3.41271977e-06,   5.24227961e-08,   1.63678798e-07])

array([['None', 'None', 'None', ..., 'None', 'None', 'None'],
       ['None', 'None', 'None', ..., 'None', 'None', 'None'],
       ['None', 'None', 'None', ..., 'None', 'None', 'None'],
       ..., 
       ['None', 'None', 'None', ..., 'None', 'None', 'None'],
       ['None', 'None', 'None', ..., 'None', 'None', 'None'],
       ['None', 'None', 'None', ..., 'None', 'None', 'None']], dtype=object)

In [18]:
#developing the multi class multi lable RandomForest classifier and training it
forest = RandomForestClassifier(n_estimators=400,max_depth=70, min_samples_split=10,min_samples_leaf=4, random_state=1)
multi_target_forest = MultiOutputClassifier(forest, n_jobs=-1)
model_mcrf= multi_target_forest.fit(x_train, y_train,sample_weight=sample_weights)

In [23]:
y_pred=model_mcrf.predict(x_test)

In [24]:
submission[list(submission.columns.values)[1:]] = y_pred
submission.to_csv('submission_rfn.csv', index=False)

In [25]:
submitted=pd.read_csv('submission_rfn.csv')

In [26]:
for column in submitted.columns[1:]:
    submitted[column]=submitted[column].apply(lambda x: 1 if (x >=.8 ) else 0)


In [27]:
submitted.to_csv('./submission_randFn.csv',index=False)

In [29]:
tbsub=pd.read_csv('submission_randFn.csv')

In [31]:
tbsub

,ID,3.1.1,3.1.2,3.2.1,3.2.2,3.3.1,3.3.2,3.3.3,3.3.4,3.3.5,...,3.8.2,3.9.1,3.9.2,3.9.3,3.a.1,3.b.1,3.b.2,3.b.3,3.c.1,3.d.1
0,11437,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,11474,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,11475,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,11476,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,11486,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,11489,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,11490,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,11494,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,11512,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,208743,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
